In [ ]:
# Feature extraction code
from skimage.feature import hog, local_binary_pattern
from skimage.measure import regionprops, label
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd
import os
import matplotlib.pyplot as plt

# Define categories
categories = ["no_tumor", "glioma_tumor", "meningioma_tumor", "pituitary_tumor"]

# extracting tumor area via thresholding and connected components
# returns area of largest connected component (aka likely the tumor)
def extract_tumor_area(image, threshold=30):
    # convert to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # applying threshold to segment the image
    _, binary = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    
    # find connected components
    labeled_img = label(binary)
    regions = regionprops(labeled_img)
    
    if not regions:
        return 0

    return max(region.area for region in regions)

# extracting HOG features
# captures shape and edge information from the image
def extract_hog_features(image):
    # convert to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                  cells_per_block=(2, 2), visualize=False)
    return features

# extracting LBP features
# captures texture information from the image
def extract_lbp_features(image, P=8, R=1):
    # convert to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    # compute LBP
    lbp = local_binary_pattern(image, P, R, method='uniform')
    # create histogram
    hist, _ = np.histogram(lbp, bins=P+2, range=(0, P+2), density=True)
    return hist

# extracting statistical features
# captures distribution characteristics of pixel values
def extract_statistical_features(image):
    # convert to grayscale if it's a color image
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    features = []
    # mean intensity
    features.append(np.mean(image))
    # standard deviation
    features.append(np.std(image))
    # skewness
    features.append(np.mean(((image - np.mean(image)) / np.std(image))**3))
    # kurtosis
    features.append(np.mean(((image - np.mean(image)) / np.std(image))**4) - 3)
    # min and max values
    features.append(np.min(image))
    features.append(np.max(image))
    # median
    features.append(np.median(image))
    # 25th and 75th percentiles
    features.append(np.percentile(image, 25))
    features.append(np.percentile(image, 75))
    return features

# extract features from all images
# combines multiple feature types and applies PCA if requested
def extract_all_features(X, y, use_pca=True, n_components=100):
    print("Extracting features from images...")
    all_features = []
    tumor_areas = []
    
    for i in tqdm(range(len(X))):
        image = X[i]  # use the image directly
        
        # extract tumor area
        tumor_area = extract_tumor_area(image)
        tumor_areas.append(tumor_area)
        
        # extract HOG features
        hog_features = extract_hog_features(image)
        
        # extract LBP features
        lbp_features = extract_lbp_features(image)
        
        # extract statistical features
        stat_features = extract_statistical_features(image)
        
        # combine all features
        combined_features = np.concatenate([hog_features, lbp_features, stat_features, [tumor_area]])
        all_features.append(combined_features)
    
    # convert to numpy array
    feature_array = np.array(all_features)
    
    # apply PCA if requested
    if use_pca and feature_array.shape[1] > n_components:
        print(f"Applying PCA to reduce dimensions from {feature_array.shape[1]} to {n_components}")
        pca = PCA(n_components=n_components)
        feature_array = pca.fit_transform(feature_array)
        print(f"Explained variance ratio: {sum(pca.explained_variance_ratio_):.4f}")
    
    # create array for tumor areas
    tumor_areas = np.array(tumor_areas)
    
    return feature_array, tumor_areas

# apply feature extraction to the data
X_features, tumor_areas = extract_all_features(train_img, train_labels_encoded, use_pca=True, n_components=100)

print(f"Extracted features shape: {X_features.shape}")
print(f"Tumor areas shape: {tumor_areas.shape}")

# Check which labels are present in the data
unique_labels = np.unique(train_labels_encoded)
print(f"Unique labels in the dataset: {unique_labels}")

# Check the mapping of labels to categories
categories = ['no_tumor', 'glioma_tumor', 'meningioma_tumor', 'pituitary_tumor']
present_categories = [categories[i] for i in unique_labels]
print(f"Categories present in the dataset: {present_categories}")

# analyze relationship between tumor area and class
plt.figure(figsize=(10, 6))
for i in unique_labels:
    plt.scatter(np.where(train_labels_encoded == i)[0], tumor_areas[train_labels_encoded == i], 
                alpha=0.5, label=categories[i])
plt.xlabel('Sample Index')
plt.ylabel('Tumor Area (pixels)')
plt.title('Tumor Area by Class')
plt.legend()
plt.show()

# box plot of tumor areas by class
plt.figure(figsize=(10, 6))
sns.boxplot(x=[categories[label] for label in train_labels_encoded], y=tumor_areas)
plt.xlabel('Tumor Type')
plt.ylabel('Tumor Area (pixels)')
plt.title('Distribution of Tumor Areas by Class')
plt.show()

# Print some basic statistics about the features
print(f"Feature statistics:")
print(f"Mean feature values: {np.mean(X_features, axis=0)[:5]}... (first 5 values)")
print(f"Min feature values: {np.min(X_features, axis=0)[:5]}... (first 5 values)")
print(f"Max feature values: {np.max(X_features, axis=0)[:5]}... (first 5 values)")
print(f"Std dev of features: {np.std(X_features, axis=0)[:5]}... (first 5 values)")

# Print statistics about tumor areas by class
print("\nTumor area statistics by class:")
for i in unique_labels:
    class_areas = tumor_areas[train_labels_encoded == i]
    print(f"{categories[i]}:")
    print(f"  Count: {len(class_areas)}")
    print(f"  Mean area: {np.mean(class_areas):.2f} pixels")
    print(f"  Min area: {np.min(class_areas)}")
    print(f"  Max area: {np.max(class_areas)}")
    print(f"  Std dev: {np.std(class_areas):.2f} pixels")

# analyze class distribution
plt.figure(figsize=(10, 6))
sns.countplot(x=[categories[label] for label in train_labels_encoded])
plt.xlabel('Tumor Type')
plt.ylabel('Count')
plt.title('Class Distribution in Dataset')
plt.show()

# split data into training and validation sets (just for analysis, not saving yet)
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val, tumor_areas_train, tumor_areas_val = train_test_split(
    X_features, train_labels_encoded, tumor_areas, test_size=0.2, random_state=42, stratify=train_labels_encoded
)

print(f"\nTraining set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")

# Check class distribution in train and validation sets
train_class_counts = np.bincount(y_train)
val_class_counts = np.bincount(y_val)
print("\nClass distribution in training set:")
for i in unique_labels:
    print(f"  {categories[i]}: {train_class_counts[i]} samples")
print("\nClass distribution in validation set:")
for i in unique_labels:
    print(f"  {categories[i]}: {val_class_counts[i]} samples")

# analyze feature importance for tumor area
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# reshape tumor areas for sklearn
tumor_areas_train_reshaped = tumor_areas_train.reshape(-1, 1)
tumor_areas_val_reshaped = tumor_areas_val.reshape(-1, 1)

# train logistic regression on just tumor area
lr_tumor = LogisticRegression(max_iter=1000)
lr_tumor.fit(tumor_areas_train_reshaped, y_train)

# evaluate
y_pred_tumor = lr_tumor.predict(tumor_areas_val_reshaped)
tumor_only_accuracy = accuracy_score(y_val, y_pred_tumor)

print(f"\nAccuracy using only tumor area: {tumor_only_accuracy:.4f}")
print("\nClassification Report using only tumor area:")
print(classification_report(y_val, y_pred_tumor, target_names=present_categories))

# visualize decision boundaries based on tumor area
plt.figure(figsize=(10, 6))
# Create a scatter plot of tumor areas vs. labels
plt.scatter(tumor_areas_val, y_val, c=y_val, cmap='viridis', alpha=0.6)
plt.xlabel('Tumor Area')
plt.ylabel('Tumor Type')
plt.title('Tumor Type vs Tumor Area')
plt.yticks(range(len(present_categories)), present_categories)
plt.colorbar(ticks=range(len(present_categories)), label='Tumor Type')
plt.tight_layout()
plt.show()

# correlation analysis between features and labels
from sklearn.feature_selection import mutual_info_classif

# creating a small subset of the original features without PCA for analysis
n_samples = min(500, len(train_img))  # Use a smaller subset for faster computation
subset_indices = np.random.choice(len(train_img), n_samples, replace=False)
subset_X = train_img[subset_indices]
subset_y = train_labels_encoded[subset_indices]

# extracting a smaller set of interpretable features
interpretable_features = []
feature_names = ["Mean Intensity", "Std Deviation", "Tumor Area", "Edge Density"]

for i in range(n_samples):
    img = subset_X[i]
    # Convert to grayscale
    if len(img.shape) == 3:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        img_gray = img
        
    # Extract simple statistical features
    features = []
    # Mean intensity
    features.append(np.mean(img_gray))
    # Standard deviation
    features.append(np.std(img_gray))
    # Tumor area
    features.append(extract_tumor_area(img))
    # Edge density (using Canny edge detector)
    edges = cv2.Canny(img_gray, 100, 200)
    features.append(np.sum(edges) / (img_gray.shape[0] * img_gray.shape[1]))
    
    interpretable_features.append(features)

interpretable_features = np.array(interpretable_features)

# calculating mutual information between features and labels
mi_scores = mutual_info_classif(interpretable_features, subset_y)
mi_scores = pd.Series(mi_scores, index=feature_names)
mi_scores = mi_scores.sort_values(ascending=False)

# plotting feature importance
plt.figure(figsize=(10, 6))
mi_scores.plot.bar()
plt.title('Feature Importance by Mutual Information with Class')
plt.xlabel('Feature')
plt.ylabel('Mutual Information Score')
plt.tight_layout()
plt.show()


# save the extracted features
os.makedirs('../data/processed', exist_ok=True)
np.save('../data/processed/X_features.npy', X_features)
np.save('../data/processed/tumor_areas.npy', tumor_areas)
np.save('../data/processed/y_labels.npy', train_labels_encoded)

# save the split datasets
np.save('../data/processed/X_train.npy', X_train)
np.save('../data/processed/X_val.npy', X_val)
np.save('../data/processed/y_train.npy', y_train)
np.save('../data/processed/y_val.npy', y_val)
np.save('../data/processed/tumor_areas_train.npy', tumor_areas_train)
np.save('../data/processed/tumor_areas_val.npy', tumor_areas_val)

